# Importacoes

In [ ]:
import tensorflow as tf


In [ ]:
print(tf.__version__)


In [ ]:
import requests
import pandas as pd
import numpy as np


# Conectando com servidor

In [ ]:
diaInicio = '20210301'
diaFinal = '20220831'
idGrupo = '12'
idRegra = '102'
idTipo = 'null'
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(url)


In [ ]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)


In [ ]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')


In [ ]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape


In [ ]:
Y = np.stack(df['campoY'].values)
Y = Y.reshape(-1,1)
Y.shape


# Dados Validacao

In [ ]:
diaInicio = '20220831'
diaFinal = '20220831'
idGrupo = '12'
idRegra = '102'
idTipo = 'null'
urlValida = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(urlValida)


In [ ]:
r = requests.get(urlValida)
j = r.json()
dfValida = pd.DataFrame.from_dict(j)


In [ ]:
dfValida.head()


In [ ]:
dfTratadoValidaX = dfValida.apply(trataLinhaX, axis='columns')
XValida = np.stack(dfTratadoValidaX.values)
XValida.shape


In [ ]:
YValida = np.stack(dfValida['campoY'].values)
YValida = YValida.reshape(-1,1)
YValida.shape


# Minha Rede

## Estrutra

In [ ]:
entrada_principal = tf.keras.layers.Input(name='entrada_principal',shape=(300,1))
conv_1_principal = tf.keras.layers.Conv1D(name='conv_1_principal',filters=64, kernel_size=3, activation='relu')(entrada_principal)
conv_2_principal = tf.keras.layers.Conv1D(name='conv_2_principal',filters=64, kernel_size=3, activation='relu')(conv_1_principal)
dropout_principal = tf.keras.layers.Dropout(name='dropout_principal',rate=0.5) (conv_2_principal)
pooling_principal = tf.keras.layers.MaxPooling1D(name='pooling_principal',pool_size=2) (dropout_principal)
flatten_principal = tf.keras.layers.Flatten(name='flatten_principal') (pooling_principal)
dense_principal = tf.keras.layers.Dense(name='dense_principal',units=100, activation='relu')(flatten_principal)

entrada_secundaria = tf.keras.layers.Input(name='entrada_secundaria',shape=(900,1))
conv_1_secundaria = tf.keras.layers.Conv1D(name='conv_1_secundaria',filters=64, kernel_size=3, activation='relu')(entrada_secundaria)
conv_2_secundaria = tf.keras.layers.Conv1D(name='conv_2_secundaria',filters=64, kernel_size=3, activation='relu')(conv_1_secundaria)
dropout_secundaria = tf.keras.layers.Dropout(name='dropout_secundaria',rate=0.5) (conv_2_secundaria)
pooling_secundaria = tf.keras.layers.MaxPooling1D(name='pooling_secundaria',pool_size=2) (dropout_secundaria)
flatten_secundaria = tf.keras.layers.Flatten(name='flatten_secundaria') (pooling_secundaria)
dense_secundaria = tf.keras.layers.Dense(name='dense_secundaria', units=100, activation='relu')(flatten_secundaria)

concat = tf.keras.layers.Concatenate(name='concat')([dense_principal, dense_secundaria])
output = tf.keras.layers.Dense(name='output',units=1, activation='sigmoid')(concat)

model = tf.keras.Model(inputs=[entrada_principal, entrada_secundaria], outputs=[output])
model.summary()


## Compilacao

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=1e-4),
                metrics=['accuracy'])


## Fit - Treino

In [ ]:
X_primaria =  X1[...,None]
XValida_primaria =  XValida1[...,None]

X_secundaria =  X2[...,None]
XValida_secundaria =  XValida2[...,None]

history = model.fit(x=[X_primaria,X_secundaria],
                    y=Y, 
                    validation_data=([XValida_primaria,XValida_secundaria], YValida) ,
                    epochs=100,
                    verbose=2)


## Salva Rede

In [ ]:
path='pesos/treino_rede760.h5'
model.save(path)


# Enviando Rede

In [ ]:
!pip install paramiko


In [ ]:
import os
import paramiko


In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("191.252.219.136", username="root", password="Digicom2004$")
sftp = ssh.open_sftp()
sftp.put("pesos/treino_rede760.h5", "/root/executa-rede/redes/treino_rede760.h5")
sftp.close()
ssh.close()


# Registra Treinamento

In [ ]:
#import requests
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/executouTreinamento'
myobj = {'id': 760}
x = requests.post(url, json = myobj)
print(x.text)
